<a href="https://colab.research.google.com/github/AvellinaLeong/NHM-Nannofossil-Segmentation-Project/blob/main/test_images_visualise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## See how model predicts on test images

download all test images to evaluate performance

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Set script location to own development space
MY_DEVELOPMENT_SPACE = '/content/drive/MyDrive/development/avellina/'
import os
os.chdir(MY_DEVELOPMENT_SPACE)
!pwd
!ls

Mounted at /content/drive/
/content/drive/MyDrive/development/avellina
detectron2  Detectron2_notebooks  Mask-RCNN  Morphometrics  output


In [ ]:
project_dir = "/content/drive/MyDrive/data/species_53"

## Load detectron and model

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 6.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.0 MB/s 

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [ ]:
# Import Detectron2 and Detectron2 Logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
# Import Detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog, detection_utils as utils, build_detection_train_loader
from detectron2.structures import BoxMode

In [ ]:
import yaml

## Load configuration and test dataset


In [ ]:
# Load the saved configuration from the YAML file
config_yaml_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1/config_1.yaml"  # Path to your saved config file
cfg = get_cfg()

# Set default values for keys that might be missing in the config file
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = ""
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.TEST.EVAL_PERIOD = 500
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Merge the configuration file with the default (debugged this way)
cfg.merge_from_file(config_yaml_path)
# Initialize the predictor with the loaded configuration
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "1", "model_final_1.pth")

In [ ]:
# Create the predictor
predictor = DefaultPredictor(cfg)

[06/16 23:01:17 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/data/species_53/Detectron2_Models/1/model_final_1.pth ...


In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_test", {}, "/content/drive/MyDrive/data/species_53/data/test/coco_test.json", "/content/drive/MyDrive/data/species_53/data/test")

In [ ]:
test_metadata = MetadataCatalog.get("my_dataset_test")
test_dataset_dicts = DatasetCatalog.get("my_dataset_test")

[06/16 23:01:36 d2.data.datasets.coco]: Loaded 96 images in COCO format from /content/drive/MyDrive/data/species_53/data/test/coco_test.json


## Segment test images and save to folder

In [ ]:
# Create the output directory for segmented test images
output_dir = "/content/drive/MyDrive/data/species_53/segmented_test_images"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
import cv2

In [ ]:
from detectron2.utils.visualizer import Visualizer, ColorMode

In [ ]:
# Visualize and save segmentation on all test images
for d in test_dataset_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)

    # Visualize the prediction
    v = Visualizer(im[:, :, ::-1],
                   metadata=test_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW
    )

    # Hide labels and boxes
    v._default_font_size = 0  # Effectively hide labels by setting font size to 0

    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Convert BGR image back to RGB
    segmented_image = out.get_image()[:, :, ::-1]

    # Construct the output file path
    file_name = os.path.basename(d["file_name"])
    output_path = os.path.join(output_dir, file_name)

    # Save the image
    cv2.imwrite(output_path, segmented_image)

In [ ]:
# Construct the output file path
file_name = os.path.basename(d["file_name"])
output_path = os.path.join(output_dir, file_name)

# Save the image
cv2.imwrite(output_path, segmented_image)

True